In [3]:
from src.train import objective
from src.utils import write_json

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
def get_example_configs():
    exp_name = 'basic_example_2'
    root_dir = f"./outputs/{exp_name}"
    objective_metric = 'train_accuracy'
    objective_mode = "max"
    configs = {
        # init args pairs
        'dataloader': {
            'wce.WCEStandardNpyDataloader': {
                'dataset_dir': './data/std_npy',
                'batch_size': 32,
                'num_workers': 2,
                'drop_last': True,
        }},
        'model': {
            'wce.WCETransferModel':{
                'base_model': 'alexnet', 
                'num_proj': 2, 
                'hid_feature': 128, 
                'hid_bias': False,
                'hid_activation': 'relu',
        }},
        'pl_module': {
            'train.SupervisedClfModule': {
                'optim_name': 'adam',
                'lr': 1e-4,
        }},
        # 'logger': {
        #     'train.CSVLogger': {
        #         'save_dir': root_dir, 
        #         'name':"csv_log",
        # }},
        # 'callbacks': {
        #     'list': { 
        #         1: {'train.EarlyStopping': {
        #                 'monitor':objective_metric, 
        #                 'mode': objective_mode,
        #             }
        #         }
        #     }
        # },
        #### pl.Trainer args
        'default_root_dir': root_dir,
        'max_epochs':2,
        # 'accelerator':"gpu", 
        # 'devices':-1,
        # 'fast_dev_run':False,
        #### others
        'objective_metric': objective_metric,
    }
    return configs

In [29]:
config = get_example_configs()
results = objective(**config)
print(results)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: outputs/basic_example_2/lightning_logs

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | WCETransferModel   | 61.2 M
1 | lost_func | CrossEntropyLoss   | 0     
2 | metric    | MulticlassAccuracy | 0     
-------------------------------------------------
61.2 M    Trainable params
0         Non-trainable params
61.2 M    Total params
244.917   Total estimated model params size (MB)


Epoch 0: 100%|██████████| 100/100 [04:03<00:00,  2.43s/it, v_num=0]        
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 100/100 [03:53<00:00,  2.34s/it, v_num=0]     
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  40%|████      | 25/62 [00:15<00:22,  1.65it/s]

(raylet) [2023-06-15 18:14:01,010 E 749 749] (raylet) node_manager.cc:3069: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 15ed7593e08c4814b0b6a3cab782ecf59902ea3f8e00e83c34b96460, IP: 169.255.255.2) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 169.255.255.2`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.



Epoch 1: 100%|██████████| 100/100 [04:31<00:00,  2.72s/it, v_num=0]     

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 100/100 [04:39<00:00,  2.80s/it, v_num=0]
1.0


In [26]:
import os

os.listdir('./outputs/example/csv_log/version_0/checkpoints')

['epoch=1-step=200.ckpt']

In [18]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [19]:
def get_raytune_config():
    exp_name = 'hyperoptim_example'
    root_dir = f"./outputs/{exp_name}"
    objective_metric = 'val_loss'
    objective_mode = "min"
    configs = {
        # init args pairs
        'dataloader': {
            'wce.WCEStandardNpyDataloader': {
                'dataset_dir': './data/std_npy',
                'batch_size': tune.choice([16, 32, 64]),
                'num_workers': 4,
                'drop_last': True,
                'shuffle': True
        }},
        'model': {
            'wce.WCETransferModel':{
                'base_model': 'alexnet', 
                'num_proj': tune.choice([2, 4]), 
                'hid_feature': tune.choice([64, 256]), 
                'hid_bias': tune.choice([True, False]),
                'hid_activation': 'relu',
        }},
        'pl_module': {
            'train.SupervisedClfModule': {
                'optim_name': 'adam',
                'lr': tune.choice([1e-5, 1e-4]),
        }},
        #### pl.Trainer args
        'default_root_dir': root_dir,
        'max_epochs':1,
        # 'accelerator':"gpu", 
        # 'devices':0,
        # 'fast_dev_run':True,
        #### others
        'exp_name': exp_name,
        'objective_metric': objective_metric,
        'objective_mode': objective_mode,
        'num_trials': 3,
        'cpu_per_trial':1,
        'gpu_per_trial':0,
    }
    return configs

In [24]:
def raytune_job(objective,config):

    raytune_config = {}
    raytune_config.update(config)
    
    exp_name = config['exp_name']
    objective_metric = config['objective_metric']
    objective_mode = config['objective_mode']
    num_trials = config['num_trials']
    cpu_per_trial = config.get('cpu_per_trial',1)
    gpus_per_trial = config.get('gpu_per_trial',1)
    
    
    scheduler = ASHAScheduler(
        metric=objective_metric,
        mode=objective_mode,
        max_t=config['max_epochs'],
    )

    analysis = tune.run(
        objective,
        resources_per_trial={
            "cpu": cpu_per_trial,
            "gpu": gpus_per_trial,
        },
        metric=objective_metric,
        mode=objective_mode,
        config=raytune_config,
        num_samples=num_trials,
        name=exp_name,
        scheduler=scheduler,
    )
    
    best_config = analysis.best_config

    write_json(best_config,os.path.join(args['default_root_dir'],'best_config.json'))
    
    return best_config

    


config = get_raytune_config()
results = raytune_job(objective,config)
print(results)

ValueError: Unknown argument found in the Trainable function. The function args must include a 'config' positional parameter. Any other args must be 'checkpoint_dir'. Found: []